1. Read block raster

In [2]:
from osgeo import gdal
import numpy as np

data_path = '0_data/'

block_raster = gdal.Open(data_path + 'block_raster.tif')
block_raster_band = block_raster.GetRasterBand(1)
block_raster_data = block_raster.ReadAsArray()
row_length = block_raster.RasterYSize
col_length = block_raster.RasterXSize
tif_geotrans = block_raster.GetGeoTransform()
tif_proj = block_raster.GetProjection()

In [3]:
block_max = block_raster_data.max()

In [4]:
block_list = []
for block_i in range(block_max+1):
    block_list.append([])

for row in range(row_length):
    for col in range(col_length):
        block_id = block_raster_data[row, col]
        if block_id > 0: # This block existed, whose value is not 0.
            block_list[int(block_id)].append(row)
            block_list[int(block_id)].append(col)

2. Read observations csv

In [5]:
import pandas as pd

obs_data = pd.read_csv(data_path + '2017allobs.csv')

In [6]:
obs_raster_data = np.zeros((row_length, col_length))

for obs_i in range(len(obs_data)):
    block_id = int(obs_data.iat[obs_i,1])
    observers = int(obs_data.iat[obs_i,7])
    block_coord = block_list[block_id]
    if len(block_coord) > 0:
        obs_raster_data[block_coord[0], block_coord[1]] = observers

In [7]:
# Save to tif
driver = gdal.GetDriverByName('GTiff')
obs_blocks_dataset = driver.Create(data_path + 'obs_blocks.tif', col_length, row_length, 1, gdal.GDT_Int16)
obs_blocks_dataset.SetGeoTransform(tif_geotrans)
obs_blocks_dataset.SetProjection(tif_proj)
obs_blocks_dataset.GetRasterBand(1).WriteArray(obs_raster_data)
# Save finish.
del obs_blocks_dataset